In [1]:
import numpy as np
import obi_one as obi
import os
import pandas as pd
from conntility.connectivity import ConnectivityMatrix

In [2]:
input_root = "/Users/pokorny/Data/Circuits"
cmat_root = "/Users/pokorny/Data/ConnectivityMatrices"
output_root = "../../../obi-output/extracted_small_microcircuits_volumetric"

In [3]:
# Compute population centroid
circuit = obi.Circuit(name="nbS1-O1", path=os.path.join(input_root, "nbS1-O1", "circuit_config_postfix2.json"))
c = circuit.sonata_circuit
nodes = c.nodes[circuit.default_population_name]
population_centroid = nodes.positions().mean()

In [4]:
# nbS1-O1 - Central neurons in connectivity
df_sel = pd.read_pickle("Top10CentralNeurons_Layer_Hex.pkl")  # (Contains neuron IDs relative to nbS1-HEXn)
df_opt = pd.read_pickle("Top10CentralNeurons_Layer_Hex_OptIDX.pkl")

# Load x/y/z positions from connectivity matrix and select best neuron in each layer
base_name = "nbS1"
hex_sel = "hex0"
layers = range(1, 7)
cmat = ConnectivityMatrix.from_h5(os.path.join(cmat_root, f"{base_name}-{hex_sel.upper()}", "connectivity_matrix.h5"))
ntab = cmat.vertices.set_index("node_ids")

xyz_selection = []
for _lay in layers:
    opt_idx = df_opt.loc[(hex_sel, _lay)]["pos"]
    node_id = df_sel[("hex0", _lay)].iloc[opt_idx].tolist()
    xyz_selection.append(ntab.loc[node_id][["x", "y", "z"]].astype(float).to_list())
    assert ntab.loc[node_id]["layer"] == str(_lay), "ERROR: Layer mismatch!"
xyz_selection = pd.DataFrame(xyz_selection, columns=["x", "y", "z"], index=layers) - population_centroid  # Relative to population centroid
xyz_selection.index.name = "layer"

In [5]:
# nbS1-O1 - Central neurons in space
hex_sel = "hex0"
layers = range(1, 7)

# Compute layer centers
pos_tab = nodes.get("hex0", properties=["layer", "x", "y", "z"])
layer_centers = pos_tab.groupby("layer").mean()
layer_centers.index = layer_centers.index.astype(int)
layer_centers = layer_centers.loc[layers]

# Find position of closest neurons to layer centers
xyz_selection_space = []
for _lay in layers:
    sel_idx = np.argmin(np.sum((pos_tab[["x", "y", "z"]] - layer_centers.loc[_lay])**2, 1))
    xyz_selection_space.append(pos_tab.iloc[sel_idx][["x", "y", "z"]].astype(float).to_list())
xyz_selection_space = pd.DataFrame(xyz_selection_space, columns=["x", "y", "z"], index=layers) - population_centroid  # Relative to population centroid
xyz_selection_space.index.name = "layer"

/var/folders/5m/dt9zn2xs6ksczy18674h6ryc0000gn/T/ipykernel_47511/3852920299.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  layer_centers = pos_tab.groupby("layer").mean()


In [6]:
# Set up volumetric neuron sets

# # (1) Centered around a central neurons in the network's connectivity
# N = 10
# volumetric_neuron_sets = obi.VolumetricCountNeuronSet(ox=xyz_selection["x"].to_list(), oy=xyz_selection["y"].to_list(), oz=xyz_selection["z"].to_list(), n=N,
#                                                       property_filter=obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={}))
# virtual_sources_to_ignore = [("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", "VPM") if _lay == 1 else ("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", ) for _lay in layers]
# output_path = os.path.join(output_root, "nCN")

# (2) Centered around a central neurons in space
N = 10
volumetric_neuron_sets = obi.VolumetricCountNeuronSet(ox=xyz_selection_space["x"].to_list(), oy=xyz_selection_space["y"].to_list(), oz=xyz_selection_space["z"].to_list(), n=N,
                                                      property_filter=obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={}))
virtual_sources_to_ignore = [("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", "VPM") if _lay == 1 else ("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", ) for _lay in layers]
output_path = os.path.join(output_root, "sCN")

In [6]:
# # nbS1-O1 - PLOTTING (Central in connectivity)
# import matplotlib.pyplot as plt

# # cmat_O1 = ConnectivityMatrix.from_h5(os.path.join(cmat_root, f"{base_name}-O1", "connectivity_matrix.h5"))

# # Load x/y/z positions from connectivity matrix
# base_name = "nbS1"
# plt.figure()
# for hex_sel in [f"hex{_i}" for _i in range(7)]:
#     cmat = ConnectivityMatrix.from_h5(os.path.join(cmat_root, f"{base_name}-{hex_sel.upper()}", "connectivity_matrix.h5"))
#     plt.plot(cmat.vertices["y"], cmat.vertices["z"], '.')
# # plt.plot(cmat_O1.vertices["y"], cmat_O1.vertices["z"], ',k')
# # plt.plot(xyz_selection["y"], xyz_selection["z"], "xk")
# plt.show()

# # df_sel[("hex0", 1)]

In [8]:
# # PLOTTING (Central in space)
# import matplotlib.pyplot as plt

# plt.figure()
# for _lay in layers:
#     plt.plot(pos_tab[pos_tab["layer"] == str(_lay)]["x"] - population_centroid.loc["x"], pos_tab[pos_tab["layer"] == str(_lay)]["y"] - population_centroid.loc["y"], ".")
# for _lay in layers:
#     plt.plot(xyz_selection_space.loc[_lay]["x"], xyz_selection_space.loc[_lay]["y"], "*k")
# plt.show()

In [9]:
circuit_extractions_form = obi.CircuitExtractions(
                    initialize=obi.CircuitExtractions.Initialize(
                        circuit=circuit,
                        run_validation=False,
                        do_virtual=True,
                        create_external=True,
                        virtual_sources_to_ignore=virtual_sources_to_ignore,
                    ),
                    # neuron_set=obi.PredefinedNeuronSet(node_set=["Excitatory", "Inhibitory"], random_sample=[100, 500]),
                    # neuron_set=obi.PredefinedNeuronSet(node_set=["Layer2", "Layer3", "Layer4", "Layer5", "Layer6"], random_sample=10),
                    # neuron_set=obi.IDNeuronSet(neuron_ids=node_id_lists),
                    neuron_set=volumetric_neuron_sets
)

coupled_scan = obi.CoupledScan(form=circuit_extractions_form, output_root=output_path, coordinate_directory_option="ZERO_INDEX")

In [10]:
# Display the intermediary data
coupled_scan.multiple_value_parameters(display=True)
coupled_scan.coordinate_parameters(display=True)
coupled_scan.coordinate_instances(display=True)

[2025-06-17 13:33:29,800] INFO: 
MULTIPLE VALUE PARAMETERS
[2025-06-17 13:33:29,803] INFO: initialize.virtual_sources_to_ignore: [('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical', 'VPM'), ('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',), ('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',), ('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',), ('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',), ('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',)]
[2025-06-17 13:33:29,803] INFO: neuron_set.ox: [536.7723752498928, 456.7532512197463, 313.3539132193323, 167.75904955575197, -41.48441166992461, -404.24438343671363]
[2025-06-17 13:33:29,804] INFO: neuron_set.oy: [-153.9048518204288, -120.451012424375, -84.2362471629474, -39.53695179386955, 21.394159650640404, 98.35934257433678]
[2025-06-17 13:33:29,804] INFO: neuron_set.oz: [733.6345807499231, 603.0905432415154, 434.6597197208482, 212.74385912901107, -44.34403473

[CircuitExtraction(idx=0, scan_output_root=PosixPath('.'), coordinate_output_root=PosixPath('.'), single_coordinate_scan_params=SingleCoordinateScanParams(type='SingleCoordinateScanParams', scan_params=[SingleValueScanParam(type='SingleValueScanParam', location_list=['initialize', 'virtual_sources_to_ignore'], value=('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical', 'VPM')), SingleValueScanParam(type='SingleValueScanParam', location_list=['neuron_set', 'ox'], value=536.7723752498928), SingleValueScanParam(type='SingleValueScanParam', location_list=['neuron_set', 'oy'], value=-153.9048518204288), SingleValueScanParam(type='SingleValueScanParam', location_list=['neuron_set', 'oz'], value=733.6345807499231)], nested_coordinate_subpath_str=PosixPath('.')), type='CircuitExtraction', initialize=Initialize(type='CircuitExtractions.Initialize', circuit=Circuit(type='Circuit', name='nbS1-O1', path='/Users/pokorny/Data/Circuits/nbS1-O1/circuit_config_postfix2.json', matrix_path=None

In [11]:
coupled_scan.execute(processing_method='run')

[2025-06-17 13:33:31,326] INFO: None
Extracting subcircuit from 'nbS1-O1'
Copying morphologies for population 'S1nonbarrel_neurons' (10)


Copying containerized .h5 morphologies: 100%|██████████████████████████████████████████████████| 10/10 [00:00<00:00, 880.97it/s]

Copied 10 morphologies into container (0 already existed)
Copying 8 biophysical neuron models (.hoc) for population 'S1nonbarrel_neurons' (10)
Copying mod files
Extraction DONE


Extracting subcircuit from 'nbS1-O1'
Copying morphologies for population 'S1nonbarrel_neurons' (10)


Copying containerized .h5 morphologies: 100%|██████████████████████████████████████████████████| 10/10 [00:00<00:00, 982.99it/s]

...

Copied 10 morphologies into container (0 already existed)
Copying 1 biophysical neuron models (.hoc) for population 'S1nonbarrel_neurons' (10)
Copying mod files
Extraction DONE


Extracting subcircuit from 'nbS1-O1'
Copying morphologies for population 'S1nonbarrel_neurons' (10)


Copying containerized .h5 morphologies: 100%|██████████████████████████████████████████████████| 10/10 [00:00<00:00, 911.41it/s]

Copied 10 morphologies into container (0 already existed)
Copying 2 biophysical neuron models (.hoc) for population 'S1nonbarrel_neurons' (10)
Copying mod files
Extraction DONE
[2025-06-17 13:48:20,907] INFO: create_bbp_workflow_campaign_config() not yet complete.


{0: None, 1: None, 2: None, 3: None, 4: None, 5: None}

In [38]:
# Check circuits
from bluepysnap import Circuit

for _idx, _inst in enumerate(coupled_scan.coordinate_instances()):
    cfg = coupled_scan.output_root / str(_idx) / "circuit_config.json"
    c = Circuit(cfg)
    nodes = c.nodes['S1nonbarrel_neurons']
    mtypes = np.unique(nodes.get(properties=['mtype']), return_counts=True)
    mt_hist = [f"{_c}x{_mt}" for _mt, _c in zip(*mtypes)]
    print(f"L{layers[_idx]}: {nodes.size} neurons, {c.edges["S1nonbarrel_neurons__S1nonbarrel_neurons__chemical"].size} synapses, m-types: {mt_hist}")


L1: 10 neurons, 107 synapses, m-types: ['2xL1_DAC', '2xL1_HAC', '2xL1_NGC-DA', '1xL1_NGC-SA', '3xL1_SAC']
L2: 10 neurons, 51 synapses, m-types: ['2xL2_IPC', '1xL2_TPC:A', '7xL2_TPC:B']
L3: 10 neurons, 34 synapses, m-types: ['7xL3_TPC:A', '3xL3_TPC:C']
L4: 10 neurons, 24 synapses, m-types: ['5xL4_TPC', '5xL4_UPC']
L5: 10 neurons, 56 synapses, m-types: ['6xL5_TPC:A', '3xL5_TPC:B', '1xL5_UPC']
L6: 10 neurons, 33 synapses, m-types: ['2xL6_BPC', '2xL6_IPC', '1xL6_LBC', '5xL6_TPC:A']
